<a href="https://colab.research.google.com/github/brownsloth/transformers_concepts_notebooks/blob/main/transformers_4_implement_transformer_encoder_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bertviz

In [ ]:
from torch import nn
import torch.nn.functional as F
from transformers import AutoConfig, AutoModel, AutoTokenizer
import torch
from math import sqrt
import bertviz

## 1. Build the attention head

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, embedding_dims, head_dims):
        super().__init__()
        self.query = nn.Linear(embedding_dims, head_dims)
        self.key = nn.Linear(embedding_dims, head_dims)
        self.value = nn.Linear(embedding_dims, head_dims)
        self.scale = head_dims ** 0.5

    def forward(self, hidden_state):
        Q = self.query(hidden_state)
        K = self.key(hidden_state)
        V = self.value(hidden_state)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scale
        weights = torch.softmax(scores, dim=-1)
        output = torch.matmul(weights, V)
        return output

## 2. Build the multi-attention head

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()  # <- You were missing this

        embedding_dims = config.hidden_size
        num_heads = config.num_attention_heads
        head_dims = embedding_dims // num_heads  # Use integer division

        self.heads = nn.ModuleList(
            [AttentionHead(embedding_dims=embedding_dims, head_dims=head_dims)
             for _ in range(num_heads)]
        )
        self.output_linear = nn.Linear(embedding_dims, embedding_dims)

    def forward(self, hidden_state):
        # List of shape (batch, seq_len, head_dim)
        head_outputs = [h(hidden_state) for h in self.heads]

        # Concatenate along the last dimension to (batch, seq_len, embedding_dim)
        concat_output = torch.cat(head_outputs, dim=-1)

        output = self.output_linear(concat_output)
        return output

In [ ]:
### Lets test the dimensions using some config
sentence = 'Hi my name is Tarun and I like to read books.'
ckpt = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(ckpt)
config = AutoConfig.from_pretrained(ckpt)


tokens = tokenizer(sentence, return_tensors='pt', add_special_tokens=False)
input_embedding_layer = nn.Embedding(config.vocab_size, config.hidden_size)
input_embeddings = input_embedding_layer(tokens.input_ids)
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(input_embeddings)
print(attn_output)
print(attn_output.shape)

In [ ]:
from bertviz import head_view

model = AutoModel.from_pretrained(ckpt, output_attentions=True)

sentence_a = "You can call me Tarun."
sentence_b = "Please give me a call tomorrow morning."

viz_inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt')
attentions = model(**viz_inputs).attentions

sentence_b_start = (viz_inputs.token_type_ids == 0).sum(dim=1)

tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])

head_view(attentions, tokens, sentence_b_start, heads=[11])

## 3. Build the feed-forward/MLP part

In [ ]:
class FeedForward(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.linear1  = nn.Linear(config.hidden_size, config.intermediate_size)
    self.linear2  = nn.Linear(config.intermediate_size, config.hidden_size)

    self.gelu = nn.GELU()

    self.dropout = nn.Dropout(config.hidden_dropout_prob)

  def forward(self, x):
    x = self.linear1(x)
    x = self.gelu(x)
    x = self.linear2(x)
    x = self.dropout(x)
    return x

In [ ]:
ff_layer = FeedForward(config)
ff_out = ff_layer(attn_output)

ff_out.size()

## 4. Build the encoder layer

In [ ]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.layer_norm_1 = nn.LayerNorm(config.hidden_size)
    self.layer_norm_2 = nn.LayerNorm(config.hidden_size)
    self.attention = MultiHeadAttention(config)
    self.feed_forward = FeedForward(config)

  def forward(self, x):
    hidden_state = self.layer_norm_1(x)
    x = x + self.attention(hidden_state)
    x = x + self.feed_forward(self.layer_norm_2(x))
    return x

In [ ]:
encoder_layer = TransformerEncoderLayer(config)
input_embeddings.shape, encoder_layer(input_embeddings).size()

In [ ]:
inputs = tokenizer("My name is Tarun and I called Sushila!", return_tensors='pt', add_special_tokens=False)
inputs.input_ids

In [ ]:
from torch import embedding

class Embeddings(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.token_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
    self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
    self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
    self.dropout = nn.Dropout()

  def forward(self, input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(end=seq_length, dtype=torch.long).unsqueeze(0)
    token_embeddings = self.token_embeddings(input_ids)
    position_embeddings = self.position_embeddings(position_ids)
    embeddings = token_embeddings + position_embeddings
    embeddings_normed = self.layer_norm(embeddings)
    embeddings_normed_dropout = self.dropout(embeddings_normed)
    return embeddings_normed_dropout

In [ ]:
embedding_layer = Embeddings(config)
embedding_layer(inputs.input_ids).size()

## 5. Transformer Encoder with all layers

In [ ]:
class TransformerEncoder(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.embeddings = Embeddings(config)
    self.layers = nn.ModuleList([TransformerEncoderLayer(config) for _ in range(config.num_hidden_layers)])

  def forward(self, x):
    x = self.embeddings(x)
    for layer in self.layers:
      x = layer(x)
    return x

In [ ]:
encoder = TransformerEncoder(config)
encoder(inputs.input_ids).size()

## 6. Add task specific head

In [ ]:
class TransformerBasedSequenceClassifier(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.encoder = TransformerEncoder(config)
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.classifier = nn.Linear(config.hidden_size, config.num_labels)

  def forward(self, x):
    x = self.encoder(x)[:, 0, :] #select hidden state of the [CLS] token
    x = self.dropout(x)
    x = self.classifier(x)
    return x

config.num_labels = 3
cls = TransformerBasedSequenceClassifier(config)
cls(inputs.input_ids).size()

In [ ]:
cls(inputs.input_ids)